## 러스트의 심벌 테이블과 메모리

### 심벌 테이블: 
- 심벌 테이블은 컴파일 타임에 사용되는 식별자(변수 이름, 함수 이름 등)와 실제 메모리 주소를 연결하는 데이터 구조입니다. 
- 러스트 컴파일러는 심벌 테이블을 사용하여 코드에서 식별자를 참조할 때 해당 식별자에 대한 메모리 위치를 알 수 있습니다. 
- 이를 통해 변수에 값을 할당하거나 함수를 호출하는 등의 작업이 가능해집니다.

### 메모리: 
- 메모리는 프로그램 실행 중에 데이터를 저장하는 데 사용되는 영역입니다. 
- 러스트에서는 메모리를 효율적으로 관리하기 위해 정적 할당, 스택, 힙 등의 메모리 영역을 사용합니다. 
- 정적 할당은 컴파일 타임에 메모리가 할당되고, 스택은 지역 변수와 함수 호출 시 임시 데이터를 저장하는 데 사용되며, 
- 힙은 동적으로 할당된 데이터(예: Boxed 값, 벡터, 문자열 등)를 저장하는 데 사용됩니다.

### 심벌 테이블은 컴파일 타임에 사용되며, 식별자와 메모리 주소의 매핑을 유지합니다. 
- 심벌 테이블은 러스트 컴파일러에 의해 생성되고 유지되며, 런타임에는 직접적으로 사용되지 않습니다. 
- 반면, 메모리는 런타임에 실제 데이터를 저장하는 데 사용되며, 러스트 프로그램이 실행되는 동안 동적으로 할당 및 해제될 수 있습니다.

따라서, 심벌 테이블은 컴파일 타임에 사용되는 데이터 구조이고, 메모리는 실제 프로그램 실행 중에 데이터를 저장하는 데 사용되는 영역입니다.

# 1. 소유자(Owner)와 소유권(Owership) 알아보기 

- 

### 소유자 변수이고 소유권은 값에 있다.

### 변수 정의
-  소유자에게 소유권이 할당된다. 
- 변수를 정의할 때는 let과 let mut으로 지정한다. 이때 값이 할당되어 소유권을 가진다.  

### 유효범위
- 변수를 블럭단위로 정의하므로 변수의 유효범위는 해당 블럭이다. 이를 스코프라고 한다.

### 원시값은 항상 소유자를 가질 수 있다. 

In [3]:
let x = 100;
let mut y = 100;

### 레퍼런스 값은 이동이 발생해서 한번만 처리된다

In [6]:
let xx = String::from("문자열"); 

In [8]:
let a = Box::new(xx.clone());
let mut b = Box::new(xx.clone());

## 소유권 시스템의 규칙

### 소유자는 유일해야 함 
- 각 값은 해당 값의 소유자가 있습니다.
- 한 번에 하나의 소유자만이 해당 값의 소유자가 될 수 있습니다.

### 소멸되는 이유 
- 소유자는 값의 생명주기 동안 유효합니다.
- 소유자가 범위를 벗어나면 해당 값은 소멸됩니다.

### 나머지 사용은 참조에 의해 빌림
- 값에 대한 참조(빌림)는 소유권을 이전하지 않습니다.


## 1-1  소유권 시스템 :  소유자는 유일 


### 원시값의 유일성

- 변경이 불가하므로 유일한 값을 처리하기 위해서 메모리에 새롭게 복사함
- 새로운 메모리를 생성하므로 유일값을 유지 => 별도의 주소 생김  


In [10]:
fn main() {
    let x = 5;                        // x는 5의 소유자입니다.
    let y = x;                        // x의 값을 y에 대입하여 y가 5의 소유자가 됩니다.
    println!("x = {}, y = {}", x, y); // x와 y 모두 5의 소유자입니다.
}

main();

x = 5, y = 5


### 레퍼런스 값의 유일성 

- 실제 메모리에 저장된 값이 주소(포인터)이므로 다른 소유자의 변수가 관리하는 메모리로 복사
- 한번 생성된 주소값은 다른 소유자로 전달되면서 유일성을 유지 

 

In [9]:
fn main() {
    let x = String::from("hello");  // x는 String 값 "hello"의 소유자입니다.
    let y = x;                      // x의 소유권이 y로 이전됩니다.
    // println!("x = {}", x);       // 에러! x의 소유권은 이전되었기 때문에 사용할 수 없습니다.
    println!("y = {}", y);          // y는 "hello"의 소유자입니다.
}

main();

y = hello


##  소유자의 소유권을 유지기간 

- 변수가 생성되면 스코프 유지
- 변수는 모듈, 함수, 블럭 등에서 생성되면 그 범위내에서만 유일성을 유지
- 범위를 벗어나면 더 이상 사용할 수 없다는 뜻이다. 



In [4]:
fn main() {
    let s1 = String::from("hello");   // s1이 String 타입의 값을 소유.
    {
        let s2 = &s1;                      // s1의 소유권이 s2로 이동. s1은 더 이상 유효하지 않음.

        println!("{}", s2);               // s2는 여전히 값을 소유, "hello"가 출력.
    }
    
    println!("{}", s1);  
}                                     // 함수가 종료되면 s2가 스코프를 벗어나서, 소유하던 값의 메모리가 해제.

main();

hello
hello


### 변수가 지정한 스코프 단위로 소유권 유지

- 더 범위가 큰 변수를 특정 블럭 내에 변수에 할당하면 소유권이 이동
- 로컬 블럭 내에서 다시 이동을 안하면 소유권을 가진 변수는 유효범위까지 소유권을 유지한다. 


In [8]:
fn main() {
    let x = String::from("Hello");      // x는 스코프에 들어감
    if true {
        let y = x;                      // x의 소유권이 y로 이전됨
        println!("{}", y);              // "Hello" 출력
    }                                   // y가 범위를 벗어남
    println!("{}", x);                  // 컴파일 오류 발생
}

Error: borrow of moved value: `x`

### 소유권을 유지하려면 참조(빌림을 사용해야 함

In [12]:
fn main() {
    let x = String::from("Hello");       // x는 스코프에 들어감
    if true {
        let y = &x;                      // x의 값을  y가 빌림 
        println!("{}", y);               // "Hello" 출력
    }                                    // y가 범위를 벗어남
    println!("{}", x);                   // 소유권이 유지되어 X 값을 출력함
}

main();

Hello
Hello


## 2  소유권에 대한 이동 알아보기 


### 소유권 연관관계

- 스택 내부에서 저장된 값과 주소를 어떻게 관리할 지를 결정하는 것이 소유권과 연관

### 소유권 이동
- 스택 내부의 포인터(레퍼런스) 값이 복사되어 새로은 심벌과 연결되어 
- 기본 심벌을 더 이상 사용 못하도록 만든다. 
- 그래서 실제 사용할 수 있는 심벌이 새롭게 결정된다. 

## 2-1 소유권 이동 이해하기

### 값에 대한 소유권은 변수를 지정하면서 생성된다. 

- 소유권을 변수에 할당하는 것을 이동(move) 라고한다.
- 이 말은 변수에 레퍼런스를 저장하는 것이다. 
- 실제 값은 heap 메모리에 생기고 그것을 변수에 할당해서 소유권을 확정하기 때문이다.
- 예외 대상은 스택 메모리에 관리하는 원시값들이다. 이 값은 이동되면 동일한 값을 새로운 변수가 사용할 수 있다. 


In [5]:
fn main() {
    let s1: String = String::from("Hello!");
    println!(" s1= {}",s1 );
}

main();

 s1= Hello!


### 소유권에는 하나의 소유자만 있어야 한다.

- 기존 변수에 할당해서 소유권을 확정했지만 
-  또 다른 변수에 기존 변수의 값을 할당하면 소유권이 옮겨간다. 
- 이것도 이동(move)이다. 그래서 기존 변수는 수명이 종료된다. 

In [37]:
fn main() {
    let s1: String = String::from("Hello!");
    let s2: String = s1;
    println!("s2: {s2}");
    println!("s1: {s1}");
}

Error: borrow of moved value: `s1`

### 소유권이 사라진 변수를 다시 사용할 수 없다.

In [6]:
fn main() {
    let s1: String = String::from("Hello!");
    let s2: String = s1;
    println!("s2: {s2}");
    //println!("s1: {s1}");
}

main();

s2: Hello!


## 2-2 소유권을 새로 생성하기 

### 새로운 변수에 소유권을 지정하려면 복제가 필수이다. 

- 이때는 clone 메서드를 사용한다.  

In [7]:
fn main() {
    let s1: String = String::from("Hello!");
    let s2: String = s1.clone();
    println!("s2: {s2}");
    println!("s1: {s1}");
}

main();

s2: Hello!
s1: Hello!


## 2-3 변수에 소유권을 변경하도록 지정하기 

### 변수에 mut으로 가변변수를 지정하면 값에 대한 소유권이 변경

-  가변 변수는 값을 변경할 수 있어서 새로운 값을 가지므로 소유권을 변경할 수 있다. 

In [8]:
fn main() {
    let mut s1: String = String::from("Hello!");
    println!("s1: {s1}");
    s1 = String::from("World!");
    println!("s1: {s1}");
}

main();

s1: Hello!
s1: World!
